## Coopetition - Muon id classification 

S. Ek-In, C. Praz


[x] Select features and find new features \
[ ] Add Scaling to wide range variables \
[ ] CatBoost \
[x] Scale weight 

-> Need the module swifter with fsspec==0.3.3 , if the version is newer than this, the code might break



### Import part 


In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Download dataset

In [2]:
# The datasets are available in CoCalc in ~/share/data/I-coopetition-muon-id/
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

### Step 1 Load Data

In [3]:
DATA_PATH = "~/share/data/I-coopetition-muon-id/"
columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight", "sWeight", "kinWeight"]
train = utils.load_full_train_csv(DATA_PATH, None)


### Step 2 Data Preprocessing 

In [4]:
# Main Setting 

weight_name = "weight"
num_sample = 10
train_vis = train.head(num_sample)

In [5]:
'''
    Utils functions
'''

#classes = [0, 1]
#columns = ['P', 'PT','ncl[0]', 'avg_cs[0]', 'ndof', 'MatchedHit_TYPE[0]', 'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]']


def visualize(feature, target, weights, num_bins=100):
    classes = np.unique(target)
    bins = np.linspace(feature.min(), feature.max(), num_bins + 1)
    
    # Plot all class 
    for c in classes:
        selection = (target == c)
        plt.hist(feature[selection], bins = bins, label = c, alpha = 0.5, weights = weights[selection])
    plt.legend()


In [6]:
print(train.columns)

Index(['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_

In [79]:
# Looking for high level parameters 
import swifter 
def Get_closest_hits(data):
    
    closest_hits_features = data.swifter.apply(utils.find_closest_hit_per_station, result_type="expand", axis=1)
    closest_hits_features.columns = ["closest_{}".format(ind) for ind in range(len(closest_hits_features.columns))]
    
    return closest_hits_features

close_hits = Get_closest_hits(train)
train_mod = pd.concat([train, close_hits], axis = 1)

# Save to files for backing up
close_hits.to_pickle('train_closest_hit.pkl')



In [80]:
# Scale product of sWeight
import pandas as pd
pd.options.mode.chained_assignment = None 

def scale_sweight(data, edge_weight = 3):

    data_mod = data[data['weight'].abs() < edge_weight]
    data_mod['scale_weight'] = (data['weight'] + edge_weight)/(2 * edge_weight)
    data_mod['scale_weight'] = data_mod['scale_weight'] * (data_mod['weight'].sum() / data_mod['scale_weight'].sum())
    
    return data_mod

# Scale sweight
train_mod = scale_sweight(train_mod, edge_weight = 3)
    
# Add new features 
def feat_PZ(data): 
    return np.sqrt(data['P'] ** 2  - data['PT'] ** 2)

train_mod['PZ'] = feat_PZ(train_mod)

In [81]:
# Visualisation 

for column in []:
    plt.figure()
    visualize(train[column][:num_sample], train['label'][:num_sample], train[weight_name][:num_sample])
    plt.title(column)


## Note: 
 - Acc = 0.832126, train_cols with scaled sWeight, add PZ, Model: XGBoost LR = 0.1 [w/o close_hits]
 - Acc = 0.862225, train_cols with close_hits, scaled sWeight, add PZ, Model: XGBoost LR = 0.1 
 - Acc = 0.85      above with StandardScaler
 - Acc = 0.84      above with PCA

In [82]:
# Def used columns 
train_cols = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]', 'FOI_hits_N',
       'PT', 'PZ'] + close_hits.columns.tolist()
target_col = ['label', 'scale_weight']

In [83]:
import re

def prepare_data(data):
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data.columns.values]



In [84]:
# Check Heading data
train.head()
X_mod, y_mod = train_mod[train_cols], train_mod[target_col]

# Rename - ignore [] 
prepare_data(X_mod)

# Splitting 
X_train, X_val, y_train, y_val = train_test_split(X_mod, y_mod, test_size=0.25, shuffle=True, random_state=2342234)



# Scale 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
'''
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
pca = PCA()
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
'''



'\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_val = scaler.transform(X_val)\npca = PCA()\nX_train = pca.fit_transform(X_train)\nX_val = pca.transform(X_val)\n'

In [85]:
print(X_train)

       ncl_0_  ncl_1_  ncl_2_  ncl_3_  avg_cs_0_  avg_cs_1_  avg_cs_2_  \
id                                                                       
71891    73.0    13.0    12.0     8.0   2.041096   1.384615   1.083333   
32976   146.0    36.0    38.0    20.0   3.130137   3.972222   2.026316   
24461    53.0    21.0    18.0     9.0   2.301887   1.476190   1.500000   
25203    31.0    22.0    12.0    13.0   1.870968   2.045455   2.333333   
54940   151.0    15.0    10.0     9.0   3.039735   1.400000   1.600000   
...       ...     ...     ...     ...        ...        ...        ...   
96257    58.0    30.0     9.0     9.0   2.620690   1.833333   1.222222   
54384    69.0     5.0    12.0    14.0   2.449275   1.000000   1.166667   
15953   123.0    31.0    11.0    10.0   4.536585   2.258065   1.363636   
7810    118.0    11.0    13.0    10.0   2.161017   4.909091   2.230769   
99838    57.0    12.0     5.0    10.0   3.105263   2.416667   1.200000   

       avg_cs_3_  ndof  MatchedHit_TY

### Step 3 Training part 

In [86]:
# Model Arch
n_trees = 100
model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = 0.1)


In [88]:
from sklearn.metrics import accuracy_score
import joblib
# Trainning part

score_best = 0
model_best = None
lr_best    = None

lrs = [0.1]

for lr in lrs:
    print("trainning with lr = {}".format(lr))
    model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = lr)

    model.fit(X_train.iloc[:, :],
              y_train['label'].values,
              sample_weight=y_train.scale_weight.values,
              verbose=True,
             )
    ''' train with scaler and PCA 
    
    model.fit(X_train,
              y_train['label'].values,
              sample_weight=y_train.scale_weight.values,
              verbose=True,
             )
    '''
    validation_predictions = model.predict_proba(X_val)[:, 1]
    model_score = scoring.rejection90(y_val.label.values, validation_predictions, sample_weight = y_val.scale_weight.values)
    print("NN: {} , Test accuracy: {}".format(lr, model_score))
    if model_score > score_best :
        model_best = model
        score_best = model_score
        lr_best    = lr

# Save Model 
joblib.dump(model_best, 'Model_Best.pkl')


trainning with lr = 0.1


NN: 0.1 , Test accuracy: 0.8622258183427993


['Model_Best.pkl']

In [89]:
print(score_best)

0.8622258183427993


In [90]:
validation_predictions = model.predict_proba(X_val)[:, 1]
model_score = scoring.rejection90(y_val.label.values, validation_predictions, sample_weight = y_val.scale_weight.values)

In [91]:
print(model_score)

0.8622258183427993


### Step 4 Predict on the whole test set and prepare submission



In [92]:

# Read data

test = utils.load_full_test_csv(DATA_PATH, None)


# Transform data 
close_hits_test = Get_closest_hits(test)
test_mod = pd.concat([test, close_hits_test], axis = 1)

test_mod['PZ'] = feat_PZ(test_mod)
X_test = test_mod[train_cols]
prepare_data(X_test)


# Predict and save file 
predictions = model_best.predict_proba(X_test)[:, 1]

compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)